<a href="https://colab.research.google.com/github/summermccune/Tokenization-Testing-for-Malware-Data/blob/main/Word2Vec/filter_unigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import glob
import nltk
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC
from google.colab import drive
drive.mount('/content/drive')
import csv
from collections import Counter
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
def count(df, c):
  for row in df['Opcodes']:
     data = row.split()
     c.update(data)

In [100]:
def removeNonVocab(vocab, series):
  rows = []
  vocab_str = '|'.join(vocab)
  pattern = '\\b((?!\\b( |' + vocab_str + ')\\b).)*\\b'
  for row in series:
    row = re.sub(pattern, '', row)
    row = re.sub(' +', ' ', row)
    rows.append(row)
  return rows

In [101]:
from os import remove
#read in the data
FakeRean = pd.read_csv('/content/drive/My Drive/Data/FakeRean.csv')
OnLineGames = pd.read_csv('/content/drive/My Drive/Data/OnLineGames.csv')
Vobfus = pd.read_csv('/content/drive/My Drive/Data/Vobfus.csv')
Winwebsec = pd.read_csv('/content/drive/My Drive/Data/Winwebsec.csv')

#splitting into test and train
FakeRean_train, FakeRean_test = train_test_split(FakeRean, test_size=0.2, random_state = 7)
OnLineGames_train, OnLineGames_test = train_test_split(OnLineGames, test_size=0.2, random_state = 7)
Vobfus_train, Vobfus_test = train_test_split(Vobfus, test_size=0.2, random_state = 7)
Winwebsec_train, Winwebsec_test = train_test_split(Winwebsec, test_size=0.2, random_state = 7)

#converting opcodes to unigram list
unigramList = []
countTotal = Counter()
count(FakeRean_train, countTotal)
count(OnLineGames_train,countTotal)
count(Vobfus_train,countTotal)
count(Winwebsec_train,countTotal)

print(countTotal)

total_count = countTotal.most_common(31)
countList = [x[0] for x in total_count]
print(countList)

#create empty df
df = pd.DataFrame(columns = ['Opcodes', 'MalwareType'])

FakeRean_rows = removeNonVocab(countList, FakeRean_train['Opcodes'])
for row in FakeRean_rows:
  df.loc[len(df)] = [row, 0]

OnLineGames_rows = removeNonVocab(countList, OnLineGames_train['Opcodes'])
for row in OnLineGames_rows:
  df.loc[len(df)] = [row, 1]

Vobfus_rows = removeNonVocab(countList, Vobfus_train['Opcodes'])
for row in Vobfus_rows:
  df.loc[len(df)] = [row, 2]

Winwebsec_rows = removeNonVocab(countList, Winwebsec_train['Opcodes'])
for row in Winwebsec_rows:
  df.loc[len(df)] = [row, 3]

df.head()

#make df to csv
df.to_csv('cleaned.csv')


#turn count into csv
total_count_df = pd.DataFrame(total_count)
total_count_df.to_csv('total_count.csv')

Counter({'add': 23087854, 'mov': 20429050, 'push': 15445942, 'pop': 7519450, 'inc': 5810925, 'call': 5007533, 'xchg': 4964945, 'or': 4788317, 'dec': 4772277, 'cmp': 4261939, 'xor': 4151652, 'sub': 4086212, 'and': 4018088, 'adc': 3994182, 'lea': 3790175, 'sbb': 3762405, 'test': 2621933, 'out': 2252309, 'in': 2205548, 'jmp': 2043840, 'movl': 1864469, 'int3': 1574178, 'ret': 1425302, 'je': 1399473, 'imul': 1377532, 'nop': 1209677, 'lods': 1102820, 'stos': 1097239, 'scas': 1071095, 'lret': 1068851, 'jne': 1060458, 'insb': 912964, 'cmc': 891876, 'jb': 834975, 'std': 825663, 'popa': 774089, 'ljmp': 770770, 'jae': 768285, 'pushl': 753628, 'jo': 751014, 'lcall': 733824, 'outsl': 714094, 'cld': 700213, 'outsb': 698974, 'jbe': 698971, 'js': 681023, 'hlt': 651916, 'pusha': 648785, 'arpl': 641284, 'jno': 627153, 'sti': 626837, 'jge': 621735, 'ja': 621596, 'leave': 619184, 'lock': 612243, 'jl': 612178, 'clc': 602528, 'insl': 599616, 'incl': 592828, 'movsl': 586793, 'loopne': 583840, 'das': 583442, 